In [1]:
import numpy as np
import pandas as pd
import geocoder
from requests import Session

In [2]:
# Load the cleaned CSV from the cell above
# Only run the subsequent cells if you want to gather fresh data
numbeo_beer = pd.read_table('numbeo_beer.txt', names=["rank", "city", "beer"], index_col="rank")
numbeo_bread = pd.read_table('numbeo_bread.txt', names=["rank", "city", "bread"], index_col="rank")
numbeo_coffee = pd.read_table("numbeo_coffee.txt", names=['rank', 'city', 'coffee'], index_col="rank")
numbeo = pd.merge(numbeo_beer, numbeo_bread, how="outer", on="city")
numbeo = pd.merge(numbeo, numbeo_coffee, how="outer", on="city")

city_data = pd.read_csv('../city-data/world-cities-data.csv')
city_data['region'] = city_data['region'].str.title()

In [3]:
# create a separate country column
split = numbeo['city'].str.split(", ")
numbeo['country'] = split.str[-1]
numbeo['city'] = split.str[0]

In [4]:
# a quick look at wrong country names
city_data[city_data.country.str.contains(',')].groupby(by="country").count()

,city_ascii,region,latitude,longitude,population,iso2,iso3,admin_name,city
country,,,,,,,,,
"Bahamas, The",3,3,3,3,3,3,3,2,3
"Gambia, The",9,9,9,9,8,9,9,9,9
"Korea, North",37,37,37,37,36,37,37,37,37
"Korea, South",84,84,84,84,80,84,84,84,84
"Micronesia, Federated States Of",5,5,5,5,1,5,5,5,5
"Saint Helena, Ascension, And Tristan Da Cunha",2,0,2,2,0,2,2,2,2


In [5]:
# to align with city_data column names
numbeo = numbeo.rename(columns={"city": "city_ascii"})

# some data cleaning
numbeo.loc[numbeo.city_ascii.str.contains("Jeddah"), "city_ascii"] = "Jeddah"
numbeo.loc[numbeo.city_ascii.str.contains("Calicut"), "city_ascii"] = "Calicut"
numbeo.city_ascii = numbeo.city_ascii.str.replace("Saint", "St.")

city_data.loc[city_data.country.str.contains("Bahamas"), "country"] = "Bahamas"
city_data.loc[city_data.country.str.contains("Korea, South"), "country"] = "South Korea"

In [6]:
# for US cities, generate a list of unabbreviated state names
def state_unabbr(state):
    abbr = pd.read_table("us-state-abbr.tsv", names=['abbr', 'name'], header=0)
    name = abbr.loc[abbr['abbr'] == state, 'name']
    if len(name) > 0:
        return name.values[0]
    else:
        return np.nan

states = []
for i in split:
    if i[-1] == "United States":
        states.append(state_unabbr(i[1]))

In [7]:
# separate numbeo into US and non-US cities
# for US cities, add states under admin_name, then merge with city_data latitude, longitude, region, and population
numbeo_us = numbeo[numbeo['country'] == 'United States']
numbeo_us["admin_name"] = states
us_cols = ["city_ascii", 'country','region', 'latitude', 'longitude', 'population', 'admin_name']
numbeo_us = pd.merge(numbeo_us, city_data[us_cols], how='left', on=['city_ascii', 'country', 'admin_name'])

/Users/christalee/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [8]:
# for non-US cities, don't match on admin_name
numbeo_globe = numbeo[numbeo['country'] != "United States"]
globe_cols = ["city_ascii", 'country','region', 'latitude', 'longitude', 'population', "admin_name"]
numbeo_globe = pd.merge(numbeo_globe, city_data[globe_cols], how='left', on=['city_ascii', 'country'])

In [21]:
# I don't know why they aren't the same length after they're reunited
# probably some duplicated values in numbeo_globe due to ambiguities in city, country
numbeo_admin = pd.concat([numbeo_us, numbeo_globe])

# reorder columns
admin_cols = ['city_ascii', 'admin_name', 'country', 'region', 'bread', 'beer', 'coffee', 'latitude', 'longitude', 'population']
numbeo_admin = numbeo_admin.reindex(columns=admin_cols)
numbeo_admin = numbeo_admin.rename(columns={"bread": "bread_numbeo", "beer": "beer_numbeo", "coffee": "cappuccino_numbeo"})

len(numbeo) == len(numbeo_admin)

False

In [23]:
lost = numbeo_admin[np.isnan(numbeo_admin['latitude'])]
lost

,city_ascii,admin_name,country,region,bread_numbeo,beer_numbeo,cappuccino_numbeo,latitude,longitude,population
33,Arhus,NaN,Denmark,NaN,2.87,3.05,6.57,NaN,NaN,NaN
81,Penang,NaN,Malaysia,NaN,1.11,2.45,2.94,NaN,NaN,NaN
95,Cardiff,NaN,United Kingdom,NaN,1.52,2.28,3.71,NaN,NaN,NaN
106,Navi Mumbai,NaN,India,NaN,0.45,2.21,2.29,NaN,NaN,NaN
114,Heraklion,NaN,Greece,NaN,1.13,2.13,3.39,NaN,NaN,NaN
116,Newcastle upon Tyne,NaN,United Kingdom,NaN,1.11,2.12,3.71,NaN,NaN,NaN
121,Ahmedabad,NaN,India,NaN,0.53,2.08,1.69,NaN,NaN,NaN
134,Bali,NaN,Indonesia,NaN,1.57,2.01,2.11,NaN,NaN,NaN
152,Lund,NaN,Sweden,NaN,3.01,1.88,4.33,NaN,NaN,NaN
158,Patras,NaN,Greece,NaN,0.97,1.82,3.38,NaN,NaN,NaN


In [33]:
city_data[city_data['city_ascii'].str.contains("The")]

,city_ascii,country,region,latitude,longitude,population,iso2,iso3,admin_name,city
835,Theodore,Australia,Asia & Pacific,-24.9500,150.0833,438.0,AU,AUS,Queensland,Theodore
2612,Sainte-Therese,Canada,North America,45.6333,-73.8500,26025.0,CA,CAN,Quebec,Sainte-Thérèse
2727,Thetford Mines,Canada,North America,46.1000,-71.3000,25709.0,CA,CAN,Quebec,Thetford Mines
2861,The Nation / La Nation,Canada,North America,45.3500,-75.0333,12808.0,CA,CAN,Ontario,The Nation / La Nation
3046,The Blue Mountains,Canada,North America,44.4833,-80.3833,7025.0,CA,CAN,Ontario,The Blue Mountains
3111,The Pas,Canada,North America,53.8250,-101.2533,5689.0,CA,CAN,Manitoba,The Pas
7277,Thessaloniki,Greece,Europe,40.6333,22.9500,315196.0,GR,GRC,Kentrikí Makedonía,Thessaloníki
7329,Thermi,Greece,Europe,40.5472,23.0197,16004.0,GR,GRC,Kentrikí Makedonía,Thérmi
11699,The Hague,Netherlands,Europe,52.0800,4.2700,1406000.0,NL,NLD,Zuid-Holland,The Hague
11716,The Bottom,Netherlands,Europe,17.6265,-63.2497,NaN,NL,NLD,NaN,The Bottom


In [11]:
def update_row(wrong_city, right_data):
    right_data["city"] = wrong_city
    
    y = lost[lost['city_ascii'] == wrong_city]
    y = y.drop(columns=['admin_name', 'region', 'latitude', 'longitude', 'population'])
    
    merge_cols = ['city_ascii', 'admin_name', 'region', 'latitude', 'longitude', 'population', 'city']
    z = pd.merge(y, right_data[merge_cols], how='left', left_on=['city_ascii'], right_on=['city'])
    z = z.drop(columns=['city_ascii_x', 'city'])
    z = z.rename(columns={'city_ascii_y': 'city_ascii'})
    z = z.reindex(columns=admin_cols)
    
    return z

In [12]:
found = pd.DataFrame()
for i in range(len(lost)):
#     print(lost.iloc[i])
    row = lost.iloc[i]
    city = row['city_ascii']
    x = city_data[city_data.city.str.contains(city.strip("()")[-4:]) & (city_data['country'] == row[2])]
    y = city_data[city_data.city.str.contains(city.strip("()")[:4]) & (city_data['country'] == row[2])]
    if len(x) == 1:
        found = pd.concat([found, update_row(city, x)])
    elif len(y) == 1:
        found = pd.concat([found, update_row(city, y)])

/Users/christalee/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [24]:
found

,city_ascii,admin_name,country,region,bread,beer,coffee,latitude,longitude,population
0,Aarhus,Midtjylland,Denmark,Europe,2.87,3.05,6.57,56.1572,10.2107,237551.0
0,Mumbai,Mahārāshtra,India,Asia & Pacific,0.45,2.21,2.29,18.9667,72.8333,23355000.0
0,Tynemouth,North Tyneside,United Kingdom,Europe,1.11,2.12,3.71,55.0170,-1.4230,67519.0
0,Balikpapan,Kalimantan Timur,Indonesia,Asia & Pacific,1.57,2.01,2.11,-1.1489,116.9031,598043.0
0,Pattani,Pattani,Thailand,Asia & Pacific,1.19,1.77,2.34,6.8664,101.2508,44234.0
0,Bhubaneshwar,Odisha,India,Asia & Pacific,0.54,1.63,1.09,20.2644,85.8281,837737.0
0,Makati City,Makati,Philippines,Asia & Pacific,NaN,1.52,2.78,14.5500,121.0333,582602.0
0,Lucknow,Uttar Pradesh,India,Asia & Pacific,0.46,1.51,1.68,26.8470,80.9470,3382000.0
0,Erbil,Arbīl,Iraq,Middle East,0.80,1.36,1.91,36.1911,44.0094,852500.0
0,Cebu City,Cebu,Philippines,Asia & Pacific,1.35,1.17,2.61,10.3000,123.9000,922611.0


In [14]:
numbeo_admin = pd.concat([numbeo_admin, found])
numbeo_clean = numbeo_admin.dropna()

In [19]:
numbeo.to_csv("numbeo.csv")
numbeo_admin.to_csv("numbeo_admin.csv")
numbeo_clean.to_csv("numbeo_clean.csv")

In [20]:
numbeo_clean.index = list(range(len(numbeo_clean)))
numbeo_clean.to_json("numbeo_clean.json")

In [19]:
numbeo_clean

,city_ascii,admin_name,country,region,bread,beer,coffee,latitude,longitude,population
0,Richmond,Virginia,United States,North America,2.63,3.71,4.54,37.5295,-77.4756,1075798.0
1,Fort Collins,Colorado,United States,North America,2.93,3.47,4.29,40.5478,-105.0656,312666.0
2,Boston,Massachusetts,United States,North America,3.16,3.37,4.31,42.3188,-71.0846,4688346.0
3,New York,New York,United States,North America,3.78,3.29,4.82,40.6943,-73.9249,18713220.0
4,Philadelphia,Pennsylvania,United States,North America,2.83,3.26,3.92,40.0077,-75.1339,5649300.0
...,...,...,...,...,...,...,...,...,...,...
470,Lorca,Murcia,Spain,Europe,1.46,0.90,2.52,37.6833,-1.7000,94404.0
471,Sevilla,Andalusia,Spain,Europe,1.27,0.79,1.74,37.3828,-5.9731,1212045.0
472,Odesa,Odes’ka Oblast’,Ukraine,Europe,0.54,0.73,1.20,46.4775,30.7326,1010783.0
473,Kyiv,"Kyyiv, Misto",Ukraine,Europe,0.59,0.72,1.21,50.4500,30.5236,2967000.0


In [ ]:
import qwikidata.sparql

def get_city_wikidata(city, country):
    query = """
    SELECT ?city ?cityLabel ?country ?countryLabel ?population
    WHERE
    {
      ?city rdfs:label '%s'@en.
      ?city wdt:P1082 ?population.
      ?city wdt:P17 ?country.
      ?city rdfs:label ?cityLabel.
      ?country rdfs:label ?countryLabel.
      FILTER(LANG(?cityLabel) = "en").
      FILTER(LANG(?countryLabel) = "en").
      FILTER(CONTAINS(?countryLabel, "%s")).
    }
    """ % (city, country)

    res = qwikidata.sparql.return_sparql_query_results(query)
    out = res['results']['bindings'][0]
    return out

In [ ]:
# This cell takes >10min to run D:
import time

numbeo['population2'] = np.zeros(len(numbeo), dtype=int)
for city, country in zip(numbeo["city_ascii"], numbeo['country']):
    try:
        data = get_city_wikidata(city, country)
        numbeo.loc[(numbeo['city_ascii'] == city) & (numbeo['country'] == country), "population2"] = int(data['population']['value'])
    except:
        numbeo.loc[(numbeo['city_ascii'] == city) & (numbeo['country'] == country), "population2"] = np.nan
    time.sleep(1)

In [ ]:
# This cell also takes a while
numbeo["latitude2"] = np.zeros(len(numbeo))
numbeo['longitude2'] = np.zeros(len(numbeo))
with Session() as s:
    for city, country in zip(numbeo["city_ascii"], numbeo['country']):
        latlng = geocoder.osm(city + ", " + country, session=s).latlng
        if latlng:
            numbeo.loc[(numbeo['city_ascii'] == city) & (numbeo['country'] == country), 'latitude2'] = latlng[0]
            numbeo.loc[(numbeo['city_ascii'] == city) & (numbeo['country'] == country), 'longitude2'] = latlng[1]
        time.sleep(1)

In [ ]:
# For some reason, one city consistently returns None during the for loop
# while None in locations:
#     i = locations.index(None)
#     locations[i] = geocoder.osm(numbeo["city"][i]).latlng